In [ ]:

!pip install pandas numpy requests beautifulsoup4 openpyxl xlsxwriter


In [ ]:

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from IPython.display import FileLink


In [ ]:

hoje = datetime.today()
amanha = hoje + timedelta(days=1)

jogos = [
    {"mandante": "Cruzeiro", "visitante": "Flamengo"},
    {"mandante": "Palmeiras", "visitante": "Grêmio"},
    {"mandante": "São Paulo", "visitante": "Fortaleza"},
    {"mandante": "Athletico-PR", "visitante": "Internacional"}
]


In [ ]:

df = pd.DataFrame(jogos)
df["xG Ataque Mandante"] = [1.6, 1.4, 1.5, 1.3]
df["xG Defesa Mandante"] = [1.0, 1.2, 1.1, 1.3]
df["xG Ataque Visitante"] = [1.7, 1.5, 1.3, 1.4]
df["xG Defesa Visitante"] = [1.1, 1.2, 1.0, 1.1]
df["Odd Mandante"] = [2.80, 2.10, 2.40, 2.50]
df["Odd Empate"] = [3.10, 3.20, 3.00, 3.00]
df["Odd Visitante"] = [2.50, 3.20, 2.80, 2.70]


In [ ]:

def calcular_prob_ev(row):
    lambda_home = (row["xG Ataque Mandante"] + row["xG Defesa Visitante"]) / 2
    lambda_away = (row["xG Ataque Visitante"] + row["xG Defesa Mandante"]) / 2

    poisson_home = [(np.exp(-lambda_home) * (lambda_home**k)) / np.math.factorial(k) for k in range(6)]
    poisson_away = [(np.exp(-lambda_away) * (lambda_away**k)) / np.math.factorial(k) for k in range(6)]

    matrix = np.outer(poisson_home, poisson_away)

    prob_home_win = np.sum([matrix[i, j] for i in range(6) for j in range(6) if i > j])
    prob_draw = np.sum([matrix[i, i] for i in range(6)])
    prob_away_win = np.sum([matrix[i, j] for i in range(6) for j in range(6) if i < j])

    ev_home = (prob_home_win * row["Odd Mandante"]) - 1
    ev_draw = (prob_draw * row["Odd Empate"]) - 1
    ev_away = (prob_away_win * row["Odd Visitante"]) - 1

    return pd.Series([lambda_home, lambda_away, prob_home_win, prob_draw, prob_away_win, ev_home, ev_draw, ev_away])

df[["λ Mandante", "λ Visitante",
    "Prob Vitória M", "Prob Empate", "Prob Vitória V",
    "EV Vitória M", "EV Empate", "EV Vitória V"]] = df.apply(calcular_prob_ev, axis=1)


In [ ]:

arquivo = "analise_apostas_valor.xlsx"
df.to_excel(arquivo, index=False)
FileLink(arquivo)
